In [2]:
# !pip install python==3.9
!pip install torch
!pip install deepparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 61.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 70.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
# import pytorch
from deepparse.parser import AddressParser
import deepparse


In [51]:
def generate_rand_garbage(list_bloc):
    list_garbage = ['',' ','École','Bibliothèque','Musée','Bureau de poste','Banque','Hôtel','Restaurant','Café','Cinéma','Théâtre',
                    'Stade','Gymnase','Parc aquatique','Magasin de vêtements','Pharmacie','Épicerie','Salon de coiffure','Station service',
                    'Garage','Boutique de jouets','Galerie d art','Studio de danse','Salle de concert','Centre commercial','Boulangerie',
                    'Université','Poste de police','Caserne de pompiers','Église','Mosquée','Synagogue','Temple','Hôpital','Clinique','Cabinet médical',
                    'Pharmacie','Maison de retraite','Centre de réadaptation','Salle de conférence','Salle d exposition']
    garbage = random.choice(list_garbage)
    list_extension = ['', ' de ' + list_bloc[4], ' van ' + list_bloc[4]]
    extension = random.choice(list_extension)
    if garbage == '': 
        return ''
    else : 
        return garbage + extension

def del_symbols(list_bloc):
    result = []
    symbols = "!@#$%^&*()_-+={}[]|\:;'<>?,./\""
    for item in list_bloc:
        new_item = ''.join([c if c.isalnum() or c.isspace() else ' ' for c in item])
        result.append(new_item)
    return result

def generate_rand_presence(list_bloc):
    N = len(list_bloc)
    random_list = [random.choices([0,1], [0.2,0.8])[0] for _ in range(N)]
    return random_list

def generate_rand_order(list_bloc):
    N = len(list_bloc)
    unique_list = list(range(1, N + 1))
    random.shuffle(unique_list)
    return unique_list

def add_typo_error(list_bloc, list_order):
    result = []
    error_type = random.choice(['nothing','add', 'delete'])
    for i in range(len(list_bloc)):
        string = list_bloc[i]
        order = list_order[i]
        if error_type == 'nothing' or order=='PostalCode': 
            pass
        elif error_type == 'add':
            error_index = random.randint(0, len(string))
            error_char = random.choice('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
            string = string[:error_index] + error_char + string[error_index:]
        else:
            if len(string) > 0:
                error_index = random.randint(0, len(string) - 1)
                string = string[:error_index] + string[error_index+1:]
        result.append(string)
    return result

def generate_elem_category(list_bloc, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']):
    list_elem_category = []
    if len(list_bloc) != len(list_order):
        print('il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order)))
        raise 'il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order))
    for i in range(len(list_bloc)):
        bloc = list_bloc[i]
        order = list_order[i]
        len_bloc = len(bloc.split())
        elem_category = [order] * len_bloc
        list_elem_category.append(elem_category)
    return list_elem_category

def change_list_order(lst, order):
    if len(lst) != len(order):
        raise ValueError("List and order must have the same length.")
    new_lst = [None] * len(lst)
    for i, value in enumerate(lst):
        new_index = order[i] - 1  
        new_lst[new_index] = value
    return new_lst

def address_augmentation(list_bloc, list_order):
    
    list_bloc[-1] = generate_rand_garbage(list_bloc)
    # print(list_bloc)
    list_bloc = del_symbols(list_bloc)
    # print(list_bloc)
    list_bloc = add_typo_error(list_bloc, list_order)
    # print(list_bloc)
    list_elem_cat = generate_elem_category(list_bloc)
    rand_presence = generate_rand_presence(list_bloc)
    rand_order = generate_rand_order(list_bloc)
    
    list_bloc_ordered = change_list_order(list_bloc,rand_order)
    list_elem_cat_ordered = change_list_order(list_elem_cat,rand_order)
    rand_presence_ordered = change_list_order(rand_presence,rand_order)
    
    list_final_category = []
    string_final = ''
    for i in range(len(rand_presence_ordered)):
        if rand_presence_ordered[i] == 1:
            list_final_category.append(list_elem_cat_ordered[i])
            if i == 0:
                string_final = list_bloc_ordered[0]
            else :
                string_final = string_final + ' ' + list_bloc_ordered[i]
    string_final = string_final
    flattened_list_final_category = [element for sublist in list_final_category for element in sublist]
    
    return string_final, flattened_list_final_category

In [7]:
!pip install pytest
!pip install mock

In [49]:
from unittest import mock

# ['StreetName','StreetNumber','Unit','PostalCode','Municipality','GeneralDelivery']
list_bloc1 = ['86','Avenue des Camélias','','1150','Woluwé-Saint-Pierre','']
list_bloc2 = ['22','Belgielaan','','1800','Vilvoorde','']
list_bloc3 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','Boutique de jouets']
list_bloc4 = ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
list_bloc5 = ['8','Avenue de Boileau','5','1160','Auderghem','']
list_bloc6 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','']

# test part
def test_generate_rand_garbage():
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['',' de ' + list_bloc1[4]]
        g = generate_rand_garbage(list_bloc1)
        assert g == '', g
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['Bibliothèque', ' de ' + list_bloc2[4]]
        g = generate_rand_garbage(list_bloc2)
        assert g == 'Bibliothèque de Vilvoorde', g

def test_del_symbols():
    g1 = del_symbols(list_bloc1)
    assert g1 == ['86','Avenue des Camélias','','1150','Woluwé Saint Pierre',''], g1
    g2 = del_symbols(list_bloc6)
    assert g2 == ['115','Rue inventée','4','1200','Woluwé Saint Lambert',''], g2

def test_generate_rand_presence():
    g1 = generate_rand_presence(list_bloc1)
    assert sum(g1) <= 6, g1
    g2 = generate_rand_presence(list_bloc2)
    assert sum(g2) >= 0, g2

def test_generate_rand_order():
    g = generate_rand_order(list_bloc3)
    g_sorted = sorted(g)
    assert g_sorted == [1,2,3,4,5,6], g

'''
def test_add_typo_error():
    with mock.patch('random.randint') as mock_index, \
         mock.patch('random.choice') as mock_choice:  
        mock_index.return_value = 5
        mock_choice.return_value = 'delete'
        assert add_typo_error(list_bloc) == 
'''

# ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
def test_generate_elem_category():
    g = generate_elem_category(list_bloc4, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery'])
    assert g == \
        [['StreetNumber'],['StreetName','StreetName','StreetName'],['Unit'],['PostalCode'],['Municipality'],['GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery']], g

"""
def test_change_list_order():
    assert change_list_order(lst, order) == 

def test_address_augmentation():
    assert address_augmentation(list_bloc) == 
"""

test_generate_rand_garbage()
test_del_symbols()
test_generate_rand_presence()
test_generate_rand_order()
test_generate_elem_category()

In [50]:
list_add = [list_bloc1,list_bloc2,list_bloc3,list_bloc4,list_bloc5,list_bloc6]
list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']
print(list_add)
for s in list_add:    
    # print(s)
    # print(type(s))
    # print(len(s))
    print(address_augmentation(s,list_order))

[['86', 'Avenue des Camélias', '', '1150', 'Woluwé-Saint-Pierre', ''], ['22', 'Belgielaan', '', '1800', 'Vilvoorde', ''], ['115', 'Rue inventée', '4', '1200', 'Woluwé-Saint-Lambert', 'Boutique de jouets'], ['8', 'Avenue de Boileau', '5', '1160', 'Auderghem', 'Magasin de vêtements de Auderghem'], ['8', 'Avenue de Boileau', '5', '1160', 'Auderghem', ''], ['115', 'Rue inventée', '4', '1200', 'Woluwé-Saint-Lambert', '']]
(' Woluwé Saint Perre Hôtelde Woluwé Saint Pierre  8', ['Municipality', 'Municipality', 'Municipality', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'StreetNumber'])
('Viloorde  Belgilaan 1800 2 Salon de coiffue', ['Municipality', 'StreetName', 'PostalCode', 'StreetNumber', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery'])
(' 1200 15', ['PostalCode', 'StreetNumber'])
('Univesité Audergem  Avenue de Boieau ', ['GeneralDelivery', 'Municipality', 'StreetName', 'StreetName', 'StreetName'])
('5 Auderghem Avenue de Boileau Cabinet médical 1

In [ ]:
address_parser = AddressParser(model_type="bpemb", device=0)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [55]:
df_best = pd.read_csv('/kaggle/input/best-address/best.csv')
df_best.columns

# container_train = deepparse.dataset_container.ListDatasetContainer(list_train)

/tmp/ipykernel_33/3647548312.py:1: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_best = pd.read_csv('/kaggle/input/best-address/best.csv')


Index(['address_id', 'street_id', 'streetname_fr', 'streetname_nl',
       'streetname_de', 'house_number', 'box_number', 'muni_code', 'poco_code',
       'region_code', 'geom'],
      dtype='object')

In [ ]:
df_best = df_best[['streetname_fr', 'streetname_nl','house_number', 'box_number','muni_code', 'poco_code']]
